In [8]:
import sklearn
assert sklearn.__version__ >= "0.20"
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import feature_column
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras import layers
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import losses
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
seed = 49

### Generating Text Files

In [22]:
!rm -r text

In [23]:
!mkdir text
!mkdir text/clickbait
!mkdir text/normal

In [24]:
githubUrl= 'https://raw.githubusercontent.com/mtp9k/Identifying-Clickbait-SYS-60616-/main/'

In [25]:
df = pd.read_csv(f'{githubUrl}clickbait_data.csv')

In [26]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1,random_state=seed)

In [27]:
train.to_csv('clickbait_train.csv',index=False)
test.to_csv('clickbait_test.csv',index=False)

In [28]:
df = pd.read_csv('clickbait_train.csv')

In [29]:
normal = df.loc[df['clickbait']==0]['headline'].values
clickbait = df.loc[df['clickbait']==1]['headline'].values

In [30]:
def generateTensorflowTextDir(data,folderName):
    for i,line in enumerate(data):
        filename = f'{i}_{folderName}'
        with open(f"text/{folderName}/{filename}.txt", "w") as outfile:
            outfile.write(line)

In [31]:
generateTensorflowTextDir(normal,'normal')

In [32]:
generateTensorflowTextDir(clickbait,'clickbait')

### Generating Test Data

In [33]:
testData = pd.read_csv(f'{githubUrl}clickbait_test.csv')
testText = testData['headline'].values
testLabels = testData['clickbait'].values

In [34]:
tweetTestData = pd.read_csv(f'{githubUrl}tweets.csv')
tweetTestText = [' '.join(map(lambda x: x.strip("\n;[]\\"), l.split(' '))) for l in tweetTestData.postText]

tweetTestLabels = np.round(tweetTestData['isClickbait'].values)

### Evaluation Helper functions

In [35]:
from sklearn.metrics import accuracy_score
def getAccuracy(labels,predictions):
    return accuracy_score(labels,np.round(predictions))

In [36]:
def export_model(model):
    ouput = tf.keras.Sequential([
      vectorize_layer,
      model,
    ])
    return ouput

In [110]:
def runEvaluation(model,returnPredictions=False):
    model_predictions= model.predict(testText)
    model_accuracy = getAccuracy(testLabels,model_predictions)
    model_tweet_predictions = model.predict(tweetTestText)
    model_tweet_accuracy = getAccuracy(tweetTestLabels,model_tweet_predictions)
    if returnPredictions:
      return model_predictions,model_tweet_predictions
    return model_accuracy,model_tweet_accuracy
    

### Parsing Text Files

In [87]:
# Batch Size -- we set to 1 to just read it in
BATCH_SIZE = 64
DATASET_SIZE = 32000
train_size = 32000 - 6400
test_size =6400
seed = 49

In [88]:
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'text',
    batch_size=BATCH_SIZE,
    class_names=['normal','clickbait'],
    subset='training',
    validation_split=0.2, 
    seed=seed
)

Found 28800 files belonging to 2 classes.
Using 23040 files for training.


In [59]:
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to normal
Label 1 corresponds to clickbait


In [89]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
   'text/',
    batch_size=BATCH_SIZE,
    class_names=['normal','clickbait'],
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 28800 files belonging to 2 classes.
Using 5760 files for validation.


In [42]:
for x,y in raw_train_ds.take(1):
    print(x[0:10],y[0:10])

tf.Tensor(
[b'13 Ways Your Body Can Go Wrong In Space'
 b"Blizzard Entertainment's victory over bnetd sealed in Appeals Court"
 b'Tankers taken by pirates in Indian Ocean'
 b'Sorority Girls On A Selfie Spree At A Baseball Game Made People Really Mad For Some Reason'
 b"US government 'can continue' eavesdropping"
 b'19 Texts All Twentysomethings Have Sent Their Dad'
 b'Leipheimer Drops in Giro, but It Could Have Been Worse'
 b"If Celebrities' Instagrams Were On Myspace"
 b'Zimbabwe bans crop growing in urban areas'
 b'You Will Never Use Chopsticks A Different Way Again'], shape=(10,), dtype=string) tf.Tensor([1 0 0 1 0 1 0 1 0 1], shape=(10,), dtype=int32)


In [90]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return input_data
    return lowercase

In [91]:
max_features = 5000
sequence_length = 500

vectorize_layer = TextVectorization(
#     standardize=custom_standardization,
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

In [92]:
# Make a text-only dataset (without labels), then call adapt
train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)

In [93]:
def vectorize_text(text, label):
    #text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [94]:
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)

In [95]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [96]:
embedding_dim = 32

## Experiment 1: Simple NN Performance vs Traditional ML

### Base ML Models

In [97]:
def dsToNumpy(ds):
    xList=[]
    yList=[]
    for x,y in ds.unbatch():
        xList.append(x.numpy())
        yList.append(y.numpy())
    xList=np.array(xList)
    yList=np.array(yList)
    return xList,yList
x_train,y_train = dsToNumpy(train_ds)
x_val,y_val = dsToNumpy(val_ds)

In [98]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=10, random_state=seed).fit(x_train, y_train)
predictions=rf.predict(x_val)
predictions

array([1, 1, 1, ..., 0, 0, 0], dtype=int32)

In [99]:
getAccuracy(y_val,predictions)

0.8274305555555556

### Simple LSTM Model

In [100]:
def LSTM_nn():
    model = tf.keras.Sequential([
      layers.Embedding(max_features, embedding_dim,input_length=sequence_length),
      LSTM(32, return_sequences=True),
      layers.GlobalAveragePooling1D(),
      layers.Dropout(0.2),
      layers.Dense(1,activation='sigmoid')])
    return model

In [101]:
lstm_model = LSTM_nn()
optimizer = keras.optimizers.Adam(lr=0.0001)
lstm_model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])
epochs = 5
history = lstm_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/5
360/360 [==============================] - 11s 26ms/step - loss: 0.6378 - accuracy: 0.5639 - val_loss: 0.2240 - val_accuracy: 0.9436
Epoch 2/5
360/360 [==============================] - 9s 25ms/step - loss: 0.2039 - accuracy: 0.9487 - val_loss: 0.1745 - val_accuracy: 0.9578
Epoch 3/5
360/360 [==============================] - 9s 25ms/step - loss: 0.1502 - accuracy: 0.9648 - val_loss: 0.1587 - val_accuracy: 0.9637
Epoch 4/5
360/360 [==============================] - 9s 25ms/step - loss: 0.1178 - accuracy: 0.9754 - val_loss: 0.1515 - val_accuracy: 0.9661
Epoch 5/5
360/360 [==============================] - 9s 25ms/step - loss: 0.0916 - accuracy: 0.9818 - val_loss: 0.1568 - val_accuracy: 0.9653


In [102]:
export_lstm_model = export_model(lstm_model)
lstm_accuracy,lstm_tweet_accuracy = runEvaluation(export_lstm_model)

In [103]:
print('same dataset evaluation accuracy:')
print(lstm_accuracy)
print('twitter dataset evaluation accuracy:')
print(lstm_tweet_accuracy)

same dataset evaluation accuracy:
0.971875
twitter dataset evaluation accuracy:
0.5203


## Experiment 2: NN Architecture Comparisons + Ensemble

### Simple GRU Model

In [104]:
def GRU_nn():
    model = tf.keras.Sequential([
      layers.Embedding(max_features, embedding_dim,input_length=sequence_length),
      layers.GRU(32, return_sequences=True),
      layers.GlobalAveragePooling1D(),
      layers.Dropout(0.2),
      layers.Dense(1,activation='sigmoid')])
    return model

In [105]:
gru_model = GRU_nn()
optimizer = keras.optimizers.Adam(lr=0.0001)
gru_model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=['accuracy'])
epochs = 5
history = gru_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/5
360/360 [==============================] - 10s 25ms/step - loss: 0.6556 - accuracy: 0.5594 - val_loss: 0.1700 - val_accuracy: 0.9554
Epoch 2/5
360/360 [==============================] - 9s 24ms/step - loss: 0.1709 - accuracy: 0.9528 - val_loss: 0.1268 - val_accuracy: 0.9651
Epoch 3/5
360/360 [==============================] - 9s 24ms/step - loss: 0.1060 - accuracy: 0.9721 - val_loss: 0.1075 - val_accuracy: 0.9665
Epoch 4/5
360/360 [==============================] - 9s 24ms/step - loss: 0.0727 - accuracy: 0.9816 - val_loss: 0.1006 - val_accuracy: 0.9707
Epoch 5/5
360/360 [==============================] - 9s 24ms/step - loss: 0.0612 - accuracy: 0.9837 - val_loss: 0.1069 - val_accuracy: 0.9708


In [106]:
export_gru_model = export_model(gru_model)
gru_accuracy,gru_tweet_accuracy = runEvaluation(export_gru_model)

In [107]:
print('same dataset evaluation accuracy:')
print(gru_accuracy)
print('twitter dataset evaluation accuracy:')
print(gru_tweet_accuracy)

same dataset evaluation accuracy:
0.975625
twitter dataset evaluation accuracy:
0.5343


### BERT Model

In [51]:
!pip install -q tensorflow-text
!pip install -q tf-models-official

     |████████████████████████████████| 3.4MB 7.7MB/s 
     |████████████████████████████████| 1.1MB 9.2MB/s 
     |████████████████████████████████| 645kB 25.9MB/s 
     |████████████████████████████████| 706kB 36.7MB/s 
     |████████████████████████████████| 174kB 56.1MB/s 
     |████████████████████████████████| 358kB 35.4MB/s 
     |████████████████████████████████| 37.6MB 134kB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 102kB 13.0MB/s 
     |████████████████████████████████| 1.2MB 26.5MB/s 


In [52]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optmizer

In [53]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8' 

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [54]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocess_model(text_test)

In [82]:
def BERT_nn():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(1,activation='sigmoid')(net)
    return tf.keras.Model(text_input, net)

In [83]:
bert_model = BERT_nn()
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()
epochs = 2
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')
bert_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)
history = bert_model.fit(
    raw_train_ds,
    validation_data=raw_val_ds,
    epochs=epochs)

Epoch 1/2
360/360 [==============================] - 146s 388ms/step - loss: 0.1841 - binary_accuracy: 0.9262 - val_loss: 0.0887 - val_binary_accuracy: 0.9675
Epoch 2/2
360/360 [==============================] - 140s 388ms/step - loss: 0.0899 - binary_accuracy: 0.9675 - val_loss: 0.0887 - val_binary_accuracy: 0.9675


In [84]:
bert_accuracy,bert_tweet_accuracy = runEvaluation(bert_model)

In [86]:
print('same dataset evaluation accuracy:')
print(bert_accuracy)
print('twitter dataset evaluation accuracy:')
print(bert_tweet_accuracy)

same dataset evaluation accuracy:
0.9728125
twitter dataset evaluation accuracy:
0.4544


### Results

In [131]:
results = pd.DataFrame({'Model':['LSTM','GRU','BERT'],'Test Set Accuracy':[lstm_accuracy,gru_accuracy,bert_accuracy],'Twitter Set Accuracy':[lstm_tweet_accuracy,gru_tweet_accuracy,bert_tweet_accuracy]})
results

,Model,Test Set Accuracy,Twitter Set Accuracy
0,LSTM,0.971875,0.5203
1,GRU,0.975625,0.5343
2,BERT,0.972812,0.4544


### Naieve Ensemble

In [114]:
lstm_pred,lstm_tweet_pred = runEvaluation(export_lstm_model,True)
gru_pred,gru_tweet_pred = runEvaluation(export_gru_model,True)
bert_pred,bert_tweet_pred = runEvaluation(bert_model,True)

In [116]:
ensemble_predictions = (lstm_pred + gru_pred + bert_pred) / 3
ensemble_tweet_predictions = (lstm_tweet_pred + gru_tweet_pred + bert_tweet_pred) / 3
ensemble_acc = getAccuracy(testLabels,ensemble_predictions)
ensemble_tweet_acc = getAccuracy(tweetTestLabels,ensemble_tweet_predictions)
print('Ensemble Test Set Accuracy:')
print(ensemble_acc)
print('Ensemble Tweet Set Accuracy:')
print(ensemble_tweet_acc)

Ensemble Test Set Accuracy:
0.97875
Ensemble Tweet Set Accuracy:
0.5187


## Experiment 3: Performance of Label Smoothing

In [132]:
keras.backend.clear_session()
smooth_model = GRU_nn()
label_smoothing =  1
optimizer = keras.optimizers.Adam(lr=0.0001)
smooth_model.compile(loss=losses.BinaryCrossentropy(from_logits=True,label_smoothing=label_smoothing),
              optimizer='adam',
              metrics=['accuracy'])
epochs = 5
history = smooth_model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs)

Epoch 1/5
360/360 [==============================] - 10s 25ms/step - loss: 0.6932 - accuracy: 0.5098 - val_loss: 0.6931 - val_accuracy: 0.5311
Epoch 2/5
360/360 [==============================] - 9s 24ms/step - loss: 0.6931 - accuracy: 0.5177 - val_loss: 0.6931 - val_accuracy: 0.5248
Epoch 3/5
360/360 [==============================] - 9s 24ms/step - loss: 0.6931 - accuracy: 0.5265 - val_loss: 0.6931 - val_accuracy: 0.5198
Epoch 4/5
360/360 [==============================] - 9s 24ms/step - loss: 0.6931 - accuracy: 0.5142 - val_loss: 0.6931 - val_accuracy: 0.5151
Epoch 5/5
360/360 [==============================] - 9s 24ms/step - loss: 0.6931 - accuracy: 0.5108 - val_loss: 0.6931 - val_accuracy: 0.5141


In [129]:
export_smooth_model = export_model(smooth_model)
smooth_accuracy,smooth_tweet_accuracy = runEvaluation(export_smooth_model)

In [130]:
print('same dataset evaluation accuracy:')
print(smooth_accuracy)
print('twitter dataset evaluation accuracy:')
print(smooth_tweet_accuracy)

same dataset evaluation accuracy:
0.9790625
twitter dataset evaluation accuracy:
0.5327
